<h1 style="color: green">Covid Tracker</h1>
<h2 style="color: green">Recovery Data</h2>
<hr style="border: 1px solid darkgreen"/>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd